# Opentrons Protocol for AFB Epistasis Mutant Cloning

First we will import python API (Application Programming Interface) into our script so that we can develop a protol that can be understood by the robot


In [1]:
!pip install --upgrade opentrons

  Using cached urwid-1.3.1.tar.gz (588 kB)
  Using cached pyserial-3.4-py2.py3-none-any.whl (193 kB)
  Using cached aionotify-0.2.0-py3-none-any.whl (6.6 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Created wheel for urwid: filename=urwid-1.3.1-py3-none-any.whl size=220775 sha256=c82940ef69a888a8d8558acd8f7ee165f8f6b42d628bed12fc388b2afdf36047
  Stored in directory: c:\users\jonbr\appdata\local\pip\cache\wheels\de\47\5b\9bd731a08c65780b2f121f53c0aba43397a9e906f463f758ec
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3166 sha256=9bd434b1e18d2490884ee755bfc5c01010cc6e7661afa2c5762823e8939da6dd
  Stored in directory: c:\users\jonbr\appdata\local\pip\cache\wheels\6a\f5\9c\f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built urwid idna-ssl


Metadata is a dictionary of data that is read by the server and returned to the opentrons app. give yourself credit.
you are however required to specify the 'apiLevel' here

In [2]:
from opentrons import protocol_api
metadata = {
    'protocolName': 'AFB Epistasis Mutations Protocol',
    'author': 'John Bryant <jbryant2@vt.edu>',
    'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants',
    'apiLevel': '2.6'
}
print(metadata)

C:\Users\jonbr\.opentrons\deck_calibration.json not found. Loading defaults
C:\Users\jonbr\.opentrons\robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!
{'protocolName': 'AFB Epistasis Mutations Protocol', 'author': 'John Bryant <jbryant2@vt.edu>', 'description': 'Protocol for performing PCR reactions and Plasmid assembly for TIR1 and AFB mutants', 'apiLevel': '2.6'}


Protocols are structured around a function called run(protocol). The function has to be name "run" and must take exactly one mandatory argument ("protocol"). function run is a container for the code that defines your protocol. 

protocol context has 2 responsibilities:
1)remember, track, and check robot's state
2)expose the functions that make the robot execute actions.

# Before Running this code, define the following variables: 

In [1]:
#variables:
#primer dilutions:
stkprm = 100 #concentration of the stock primer you are adding
stkvol = 1 #the volume of stock primer you are adding
dilprm = 2.5 #this is the concentration in uM that you want your working dilution to be

#pcr reaction
# need to get this from the df##Numprimers = 4 #this is how many primers go in each pcr reaction.
primerconcentration = 0.1 #this is the concentration you want each primer to be in the pcr reaction
pcrvol = 25 #this is the total volume of your pcr reaction 
templatengs = .5 #this is the concentration of template you want in your pcr rxn in ng/uL

#template dilutions tells you what the temps need to be diluted to initially so that you can just add 1 uL of template to the pcr:
#need to fill in stock template values further down the script
diltemp = (templatengs)*(pcrvol)/1



In [3]:
#first import information from the j5 spreadsheet in order to perform appropriate steps
import feather
import pandas as pd
import numpy as np

#path for my imac
#path = '/Users/jbryant2/Google Drive File Stream/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/oligo.feather'

#path for my windows machine
path = '/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/oligo.feather'

oligos = feather.read_dataframe(path)
oligos

oligos['ID Number'] = oligos['ID Number'].astype(int)
oligos

,ID Number,Name,Length,Tm,Tm (3' only),Cost,Sequence,Sequence (3' only)
0,0,mas00001_(AFB1_V113A)_forward,35.0,70.153,63.883,3.5,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCG,TCAGCATCTTTTACTTTCACCAGCG
1,1,mas00002_(AFB1_V113A)_reverse,23.0,60.622,60.622,2.3,CCACCATCCTCTTCATTCTGATC,CCACCATCCTCTTCATTCTGATC
2,2,mas00003_(C)_(AFB1_V113A2)_forward,47.0,72.938,66.252,4.7,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTGC,GACTGATGAGTGCTTAGAGAAAATTGCTGC
3,3,mas00004_(AFB1_V113A2)_(C2)_reverse,32.0,74.396,67.667,3.2,GATGCTGAGGATCAGTTGGGTGCACGAGTGGG,GATCAGTTGGGTGCACGAGTGGG
4,4,mas00005_(AFB1_G143D)_reverse,24.0,59.182,59.182,2.4,CATCAGTAGAGAAACCTTCACAAG,CATCAGTAGAGAAACCTTCACAAG
5,5,mas00006_(A2)_(AFB1_G143D2)_forward,48.0,71.989,68.756,4.8,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCAGG,TATCGCTGCTATTGCAGCTACTTGCAGG
6,6,mas00007_(AFB1_K8E)_reverse,17.0,58.827,60.527,1.7,GGTGGGAATCGGAGTCC,GGTGGGAATCGGAGTCC
7,7,mas00008_(G)_(AFB1_K8E2)_forward,41.0,72.721,61.592,4.1,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCC,AGGTGTTGGAACATATCCTCTCC
8,8,mas00009_(AFB1_A460E)_reverse,28.0,63.338,63.338,2.8,CTATTGATAGCATCCTAACCTTCTTGGC,CTATTGATAGCATCCTAACCTTCTTGGC
9,9,mas00010_(_A_)_(AFB1_A460E2)_forward,51.0,69.542,68.776,5.1,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...,ATTTGCTGGGGACAGTGATTTGATGCTTCA


In [4]:
if len(oligos.columns) < 9:
    oligos['well'] = ''
    oligos['stock primer concentration'] = ''
    oligos['volume of stock primer to add'] = ''
    oligos['concentration of diluted primer'] = ''
    oligos['volume of diluted primer'] = '' #this is a calculated value
    oligos['how much of the diluted primer is left'] = '' #also a calculated value
oligos

,ID Number,Name,Length,Tm,Tm (3' only),Cost,Sequence,Sequence (3' only),well,stock primer concentration,volume of stock primer to add,concentration of diluted primer,volume of diluted primer,how much of the diluted primer is left
0,0,mas00001_(AFB1_V113A)_forward,35.0,70.153,63.883,3.5,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCG,TCAGCATCTTTTACTTTCACCAGCG,,,,,,
1,1,mas00002_(AFB1_V113A)_reverse,23.0,60.622,60.622,2.3,CCACCATCCTCTTCATTCTGATC,CCACCATCCTCTTCATTCTGATC,,,,,,
2,2,mas00003_(C)_(AFB1_V113A2)_forward,47.0,72.938,66.252,4.7,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTGC,GACTGATGAGTGCTTAGAGAAAATTGCTGC,,,,,,
3,3,mas00004_(AFB1_V113A2)_(C2)_reverse,32.0,74.396,67.667,3.2,GATGCTGAGGATCAGTTGGGTGCACGAGTGGG,GATCAGTTGGGTGCACGAGTGGG,,,,,,
4,4,mas00005_(AFB1_G143D)_reverse,24.0,59.182,59.182,2.4,CATCAGTAGAGAAACCTTCACAAG,CATCAGTAGAGAAACCTTCACAAG,,,,,,
5,5,mas00006_(A2)_(AFB1_G143D2)_forward,48.0,71.989,68.756,4.8,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCAGG,TATCGCTGCTATTGCAGCTACTTGCAGG,,,,,,
6,6,mas00007_(AFB1_K8E)_reverse,17.0,58.827,60.527,1.7,GGTGGGAATCGGAGTCC,GGTGGGAATCGGAGTCC,,,,,,
7,7,mas00008_(G)_(AFB1_K8E2)_forward,41.0,72.721,61.592,4.1,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCC,AGGTGTTGGAACATATCCTCTCC,,,,,,
8,8,mas00009_(AFB1_A460E)_reverse,28.0,63.338,63.338,2.8,CTATTGATAGCATCCTAACCTTCTTGGC,CTATTGATAGCATCCTAACCTTCTTGGC,,,,,,
9,9,mas00010_(_A_)_(AFB1_A460E2)_forward,51.0,69.542,68.776,5.1,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...,ATTTGCTGGGGACAGTGATTTGATGCTTCA,,,,,,


In [5]:
#custom 4x6 well plate dictionary. hardcoded specifically for the labware used. 
#this could easily be replace with another well specification dictionary

id2well = {}
id2well['0'] = 'A1'
id2well['1'] = 'A2'
id2well['2'] = 'A3'
id2well['3'] = 'A4'
id2well['4'] = 'A5'
id2well['5'] = 'A6'
id2well['6'] = 'B1'
id2well['7'] = 'B2'
id2well['8'] = 'B3'
id2well['9'] = 'B4'
id2well['10'] = 'B5'
id2well['11'] = 'B6'
id2well['12'] = 'C1'
id2well['13'] = 'C2'
id2well['14'] = 'C3'
id2well['15'] = 'C4'
id2well['16'] = 'C5'
id2well['17'] = 'C6'
id2well['18'] = 'D1'
id2well['19'] = 'D2'
id2well['20'] = 'D3'
id2well['21'] = 'D4'
id2well['22'] = 'D5'
id2well['23'] = 'D6'

In [6]:
for i, row in oligos.iterrows():
    oligos.loc[i,'well'] = id2well[str(i)] #this only works because the index matces the id number. id number is a floating value
    oligos.loc[i,'stock primer concentration'] = stkprm
    oligos.loc[i,'volume of stock primer to add'] = stkvol
    oligos.loc[i,'concentration of diluted primer'] = dilprm
    
for i, row in oligos.iterrows():
    oligos.loc[i,'volume of diluted primer'] = row['stock primer concentration']*row['volume of stock primer to add']/row['concentration of diluted primer']
    
oligos

,ID Number,Name,Length,Tm,Tm (3' only),Cost,Sequence,Sequence (3' only),well,stock primer concentration,volume of stock primer to add,concentration of diluted primer,volume of diluted primer,how much of the diluted primer is left
0,0,mas00001_(AFB1_V113A)_forward,35.0,70.153,63.883,3.5,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCG,TCAGCATCTTTTACTTTCACCAGCG,A1,100,1,2.5,40,
1,1,mas00002_(AFB1_V113A)_reverse,23.0,60.622,60.622,2.3,CCACCATCCTCTTCATTCTGATC,CCACCATCCTCTTCATTCTGATC,A2,100,1,2.5,40,
2,2,mas00003_(C)_(AFB1_V113A2)_forward,47.0,72.938,66.252,4.7,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTGC,GACTGATGAGTGCTTAGAGAAAATTGCTGC,A3,100,1,2.5,40,
3,3,mas00004_(AFB1_V113A2)_(C2)_reverse,32.0,74.396,67.667,3.2,GATGCTGAGGATCAGTTGGGTGCACGAGTGGG,GATCAGTTGGGTGCACGAGTGGG,A4,100,1,2.5,40,
4,4,mas00005_(AFB1_G143D)_reverse,24.0,59.182,59.182,2.4,CATCAGTAGAGAAACCTTCACAAG,CATCAGTAGAGAAACCTTCACAAG,A5,100,1,2.5,40,
5,5,mas00006_(A2)_(AFB1_G143D2)_forward,48.0,71.989,68.756,4.8,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCAGG,TATCGCTGCTATTGCAGCTACTTGCAGG,A6,100,1,2.5,40,
6,6,mas00007_(AFB1_K8E)_reverse,17.0,58.827,60.527,1.7,GGTGGGAATCGGAGTCC,GGTGGGAATCGGAGTCC,B1,100,1,2.5,40,
7,7,mas00008_(G)_(AFB1_K8E2)_forward,41.0,72.721,61.592,4.1,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCC,AGGTGTTGGAACATATCCTCTCC,B2,100,1,2.5,40,
8,8,mas00009_(AFB1_A460E)_reverse,28.0,63.338,63.338,2.8,CTATTGATAGCATCCTAACCTTCTTGGC,CTATTGATAGCATCCTAACCTTCTTGGC,B3,100,1,2.5,40,
9,9,mas00010_(_A_)_(AFB1_A460E2)_forward,51.0,69.542,68.776,5.1,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...,ATTTGCTGGGGACAGTGATTTGATGCTTCA,B4,100,1,2.5,40,


In [7]:
#path for my imac
#path = '/Users/jbryant2/Google Drive File Stream/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/pcr.feather'

#path for my windows machine
path = '/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/pcr.feather'

pcr = feather.read_dataframe(path)
pcr

pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']] = pcr[['Reaction ID Number','Forward Oligo ID Number','Reverse Oligo ID Number']].astype(int)
pcr

,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,Mean Oligo Tm (3' Only),Delta Oligo Tm (3'Only),Length,Sequence
0,0,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,1,mas00002_(AFB1_V113A)_reverse,PCR,65.3875,9.531,62.2525,3.261,2680.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
1,1,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,2,mas00003_(C)_(AFB1_V113A2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.6670,1.458,66.9595,1.415,4939.0,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTG...
2,2,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,4,mas00005_(AFB1_G143D)_reverse,PCR,64.6675,10.971,61.5325,4.701,2770.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
3,3,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,5,mas00006_(A2)_(AFB1_G143D2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.1925,2.407,68.2115,1.089,4852.0,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCA...
4,4,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,6,mas00007_(AFB1_K8E)_reverse,PCR,64.4900,11.326,62.2050,3.356,2363.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
5,5,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,7,mas00008_(G)_(AFB1_K8E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.5585,1.675,64.6295,6.075,5256.0,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCCTTCAT...
6,6,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,8,mas00009_(AFB1_A460E)_reverse,PCR,66.7455,6.815,63.6105,0.545,3721.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
7,7,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,9,mas00010_(_A_)_(AFB1_A460E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,71.9690,4.854,68.2215,1.109,3902.0,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...
8,8,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,10,mas00011_(AFB1_S434E)_reverse,PCR,67.6875,4.931,64.5525,1.339,3642.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
9,9,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,11,mas00012_(glutamic_acid)_(AFB1_S434E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,75.3405,1.889,67.7385,0.143,3977.0,GGATCTTCGACGGCTCGAGGTCTCTGGTCTTCTCTCTGACAAGGCC...


from the pcr reactions dataframe object, we need to first figure out which templates are used, have the user input the template concentrations, and then create a new column with template concentrations in this dataframe.

In [8]:
#here we create an object with each unique template from the
templates = pcr["Primary Template"]
unique_templates = templates.drop_duplicates(keep = 'first', inplace = False)
unique_templates


0     pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557
10                      pWL63_pGP8G2_TIR1_3f6h
Name: Primary Template, dtype: object

In [9]:
df = pd.DataFrame(unique_templates)

df = df.reset_index()
df = df.drop('index', 1)

df['Template Concentration'] = ''
df

,Primary Template,Template Concentration
0,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,
1,pWL63_pGP8G2_TIR1_3f6h,


The user has to come down here and enter in a vector of the template concentrations **In the correct order of the the unique_templates dataframe** the code will then automatically populate the pcr dataframe with this information

In [10]:
#enter template concentrations here
df['Template Concentration'] = [126,387] #all you have to do is input the template concentrations in the right order

df

,Primary Template,Template Concentration
0,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,126
1,pWL63_pGP8G2_TIR1_3f6h,387


In [19]:
startnum = len(oligos['well'])

for i, row in df.iterrows():
    df.loc[i,'template_well'] = id2well[str(startnum+i)]
    


for i, row in df.iterrows():
    df.loc[i,'amount of template to add'] = 1
df

,Primary Template,Template Concentration,template_well,amount of template to add
0,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,126,D4,1.0
1,pWL63_pGP8G2_TIR1_3f6h,387,D5,1.0


In [20]:
#this line of code integrates the template concentrations into the pcr df
pcr_plustemplates = pd.merge(pcr,df,on='Primary Template')
pcr_plustemplates

,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,Mean Oligo Tm (3' Only),Delta Oligo Tm (3'Only),Length,Sequence,Template Concentration,template_well,amount of template to add
0,0,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,1,mas00002_(AFB1_V113A)_reverse,PCR,65.3875,9.531,62.2525,3.261,2680.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0
1,1,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,2,mas00003_(C)_(AFB1_V113A2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.6670,1.458,66.9595,1.415,4939.0,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTG...,126,D4,1.0
2,2,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,4,mas00005_(AFB1_G143D)_reverse,PCR,64.6675,10.971,61.5325,4.701,2770.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0
3,3,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,5,mas00006_(A2)_(AFB1_G143D2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.1925,2.407,68.2115,1.089,4852.0,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCA...,126,D4,1.0
4,4,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,6,mas00007_(AFB1_K8E)_reverse,PCR,64.4900,11.326,62.2050,3.356,2363.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0
5,5,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,7,mas00008_(G)_(AFB1_K8E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.5585,1.675,64.6295,6.075,5256.0,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCCTTCAT...,126,D4,1.0
6,6,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,8,mas00009_(AFB1_A460E)_reverse,PCR,66.7455,6.815,63.6105,0.545,3721.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0
7,7,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,9,mas00010_(_A_)_(AFB1_A460E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,71.9690,4.854,68.2215,1.109,3902.0,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...,126,D4,1.0
8,8,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,10,mas00011_(AFB1_S434E)_reverse,PCR,67.6875,4.931,64.5525,1.339,3642.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0
9,9,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,11,mas00012_(glutamic_acid)_(AFB1_S434E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,75.3405,1.889,67.7385,0.143,3977.0,GGATCTTCGACGGCTCGAGGTCTCTGGTCTTCTCTCTGACAAGGCC...,126,D4,1.0


In [26]:
if len(pcr_plustemplates.columns) == 17:

    for i, row in pcr_plustemplates.iterrows():
   
        pcr_plustemplates.loc[i,'concentration of template (ng/uL)'] = diltemp

pcr_plustemplates['volume of dilute template prepared'] = pcr_plustemplates['Template Concentration']*stkvol/pcr_plustemplates['concentration of template (ng/uL)']

pcr_plustemplates


,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,Mean Oligo Tm (3' Only),Delta Oligo Tm (3'Only),Length,Sequence,Template Concentration,template_well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared
0,0,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,1,mas00002_(AFB1_V113A)_reverse,PCR,65.3875,9.531,62.2525,3.261,2680.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08
1,1,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,2,mas00003_(C)_(AFB1_V113A2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.6670,1.458,66.9595,1.415,4939.0,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTG...,126,D4,1.0,12.5,10.08
2,2,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,4,mas00005_(AFB1_G143D)_reverse,PCR,64.6675,10.971,61.5325,4.701,2770.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08
3,3,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,5,mas00006_(A2)_(AFB1_G143D2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.1925,2.407,68.2115,1.089,4852.0,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCA...,126,D4,1.0,12.5,10.08
4,4,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,6,mas00007_(AFB1_K8E)_reverse,PCR,64.4900,11.326,62.2050,3.356,2363.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08
5,5,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,7,mas00008_(G)_(AFB1_K8E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.5585,1.675,64.6295,6.075,5256.0,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCCTTCAT...,126,D4,1.0,12.5,10.08
6,6,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,8,mas00009_(AFB1_A460E)_reverse,PCR,66.7455,6.815,63.6105,0.545,3721.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08
7,7,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,9,mas00010_(_A_)_(AFB1_A460E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,71.9690,4.854,68.2215,1.109,3902.0,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...,126,D4,1.0,12.5,10.08
8,8,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,10,mas00011_(AFB1_S434E)_reverse,PCR,67.6875,4.931,64.5525,1.339,3642.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08
9,9,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,11,mas00012_(glutamic_acid)_(AFB1_S434E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,75.3405,1.889,67.7385,0.143,3977.0,GGATCTTCGACGGCTCGAGGTCTCTGGTCTTCTCTCTGACAAGGCC...,126,D4,1.0,12.5,10.08


now i will tack on the wells where the forward and reverse primers are located

In [27]:
wellinfo = oligos[['ID Number','well']]
wellinfo

wellinfo = wellinfo.rename(columns={'ID Number':'Forward Oligo ID Number'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Forward Oligo ID Number')
wellinfo = wellinfo.rename(columns={'Forward Oligo ID Number':'Reverse Oligo ID Number','well':'well2'})
pcr_plustemplates = pcr_plustemplates.merge(wellinfo, on= 'Reverse Oligo ID Number')
pcr_plustemplates

,Reaction ID Number,Primary Template,Alternate Template,Forward Oligo ID Number,Forward Oligo Name,Reverse Oligo ID Number,Reverse Oligo Name,Notes,Mean Oligo Tm,Delta Oligo Tm,...,Delta Oligo Tm (3'Only),Length,Sequence,Template Concentration,template_well,amount of template to add,concentration of template (ng/uL),volume of dilute template prepared,well,well2
0,0,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,1,mas00002_(AFB1_V113A)_reverse,PCR,65.3875,9.531,...,3.261,2680.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08,A1,A2
1,2,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,4,mas00005_(AFB1_G143D)_reverse,PCR,64.6675,10.971,...,4.701,2770.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08,A1,A5
2,4,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,6,mas00007_(AFB1_K8E)_reverse,PCR,64.4900,11.326,...,3.356,2363.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08,A1,B1
3,6,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,8,mas00009_(AFB1_A460E)_reverse,PCR,66.7455,6.815,...,0.545,3721.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08,A1,B3
4,8,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,0,mas00001_(AFB1_V113A)_forward,10,mas00011_(AFB1_S434E)_reverse,PCR,67.6875,4.931,...,1.339,3642.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...,126,D4,1.0,12.5,10.08,A1,B5
5,1,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,2,mas00003_(C)_(AFB1_V113A2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.6670,1.458,...,1.415,4939.0,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTG...,126,D4,1.0,12.5,10.08,A3,A4
6,3,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,5,mas00006_(A2)_(AFB1_G143D2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.1925,2.407,...,1.089,4852.0,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCA...,126,D4,1.0,12.5,10.08,A6,A4
7,5,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,7,mas00008_(G)_(AFB1_K8E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,73.5585,1.675,...,6.075,5256.0,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCCTTCAT...,126,D4,1.0,12.5,10.08,B2,A4
8,7,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,9,mas00010_(_A_)_(AFB1_A460E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,71.9690,4.854,...,1.109,3902.0,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...,126,D4,1.0,12.5,10.08,B4,A4
9,9,pWL65_pGP8G2_AFB1_3xF6xH_pj5_0100pNL1557,None,11,mas00012_(glutamic_acid)_(AFB1_S434E2)_forward,3,mas00004_(AFB1_V113A2)_(C2)_reverse,PCR,75.3405,1.889,...,0.143,3977.0,GGATCTTCGACGGCTCGAGGTCTCTGGTCTTCTCTCTGACAAGGCC...,126,D4,1.0,12.5,10.08,B6,A4


In [28]:
#read in assembly pieces as dataframe .... might not need this info

#path for my imac
#path = '/Users/jbryant2/Google Drive File Stream/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/assembly.feather'

#path for my windows machine
path ='/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/assembly.feather'

assembly = feather.read_dataframe(path)
assembly

,Reaction ID Number,Reaction Type,Type ID Number,Part(s),Relative Overlap Position,Extra 5' CPEC bps,Extra 3' CPEC bps,CPEC Tm Next,Overlap with next (#bps),Overlap with Next (Sequence),Overlap with Next (Sequence Reverse Compliment),Sequence Length,Sequence
0,0,PCR,0,(AFB1_V113A),-8,0,0,51.520,16.0,ATGAAGAGGATGGTGG,CCACCATCCTCTTCAT,2680.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
1,1,PCR,1,(C)_(AFB1_V113A2)_(C2),0,2,0,54.912,18.0,CAACTGATCCTCAGCATC,GATGCTGAGGATCAGTTG,4939.0,ATGAAGAGGATGGTGGCGACTGATGAGTGCTTAGAGAAAATTGCTG...
2,2,PCR,2,(AFB1_G143D),-8,3,0,52.153,19.0,GAAGGTTTCTCTACTGATG,CATCAGTAGAGAAACCTTC,2770.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
3,3,PCR,3,(A2)_(AFB1_G143D2)_(C3),0,2,0,54.912,18.0,CAACTGATCCTCAGCATC,GATGCTGAGGATCAGTTG,4852.0,GAAGGTTTCTCTACTGATGATATCGCTGCTATTGCAGCTACTTGCA...
4,4,PCR,4,(AFB1_K8E),-8,1,-1,55.701,16.0,GACTCCGATTCCCACC,GGTGGGAATCGGAGTC,2363.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
5,5,PCR,5,(G)_(AFB1_K8E2)_(_c_),0,2,0,54.912,18.0,CAACTGATCCTCAGCATC,GATGCTGAGGATCAGTTG,5256.0,GACTCCGATTCCCACCTGAGGTGTTGGAACATATCCTCTCCTTCAT...
6,6,PCR,6,(AFB1_A460E),-8,4,0,52.426,20.0,GGTTAGGATGCTATCAATAG,CTATTGATAGCATCCTAACC,3721.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
7,7,PCR,7,(_A_)_(AFB1_A460E2)_(c-),0,2,0,54.912,18.0,CAACTGATCCTCAGCATC,GATGCTGAGGATCAGTTG,3902.0,GGTTAGGATGCTATCAATAGAATTTGCTGGGGACAGTGATTTGATG...
8,8,PCR,8,(AFB1_S434E),-8,0,0,56.336,16.0,GGATCTTCGACGGCTC,GAGCCGTCGAAGATCC,3642.0,CAACTGATCCTCAGCATCTTTTACTTTCACCAGCGTTTCTGGGTGA...
9,9,PCR,9,(glutamic_acid)_(AFB1_S434E2)_(-c-),0,2,0,54.912,18.0,CAACTGATCCTCAGCATC,GATGCTGAGGATCAGTTG,3977.0,GGATCTTCGACGGCTCGAGGTCTCTGGTCTTCTCTCTGACAAGGCC...


In [29]:
#path for my imac
#path = '/Users/jbryant2/Google Drive File Stream/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/combinations.feather'

#path for my windows machine
path = '/Shared drives/PlantSynBioLab/Cloning/AFB_epistasis_muts/combinations.feather'

combinations = feather.read_dataframe(path)
combinations

combinations[['ID Number','Assembly Piece ID Number Bin 0','Assembly Piece ID Number Bin 1']] = combinations[['ID Number','Assembly Piece ID Number Bin 0','Assembly Piece ID Number Bin 1']].astype(int)
combinations

,ID Number,Name,Aseembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1
0,0,pmas00001,SLIC/Gibson/CPEC,(AFB1_V113A),0,(C)_(AFB1_V113A2)_(C2),1
1,1,pmas00002,SLIC/Gibson/CPEC,(AFB1_G143D),2,(A2)_(AFB1_G143D2)_(C3),3
2,2,pmas00003,SLIC/Gibson/CPEC,(AFB1_K8E),4,(G)_(AFB1_K8E2)_(_c_),5
3,3,pmas00004,SLIC/Gibson/CPEC,(AFB1_A460E),6,(_A_)_(AFB1_A460E2)_(c-),7
4,4,pmas00005,SLIC/Gibson/CPEC,(AFB1_S434E),8,(glutamic_acid)_(AFB1_S434E2)_(-c-),9
5,5,pmas00006,SLIC/Gibson/CPEC,(TIR1_E12K),10,(A)_(TIR1_E12K2)_(c),11
6,6,pmas00007,SLIC/Gibson/CPEC,(TIR1_M473A),12,(C_)_(TIR1_M473A2)_(C1),13
7,7,pmas00008,SLIC/Gibson/CPEC,(TIR1_D170A),14,(c__)_(TIR1_D170A2)_(c---),15
8,8,pmas00009,SLIC/Gibson/CPEC,(AFB1_noflagtag),16,(AFB1_noflagtag2)_(c--),17


In [30]:
primerlocations = pcr_plustemplates[['Reaction ID Number', 'well', 'well2']]
primerlocations

primerlocations = primerlocations.rename(columns={'Reaction ID Number':'Assembly Piece ID Number Bin 0'})
combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 0')
primerlocations = primerlocations.rename(columns={'Assembly Piece ID Number Bin 0':'Assembly Piece ID Number Bin 1'})
combinations = combinations.merge(primerlocations, on= 'Assembly Piece ID Number Bin 1')
combinations

,ID Number,Name,Aseembly Method,Part(s) Bin 0,Assembly Piece ID Number Bin 0,Part(s) Bin 1,Assembly Piece ID Number Bin 1,well_x,well2_x,well_y,well2_y
0,0,pmas00001,SLIC/Gibson/CPEC,(AFB1_V113A),0,(C)_(AFB1_V113A2)_(C2),1,A1,A2,A3,A4
1,1,pmas00002,SLIC/Gibson/CPEC,(AFB1_G143D),2,(A2)_(AFB1_G143D2)_(C3),3,A1,A5,A6,A4
2,2,pmas00003,SLIC/Gibson/CPEC,(AFB1_K8E),4,(G)_(AFB1_K8E2)_(_c_),5,A1,B1,B2,A4
3,3,pmas00004,SLIC/Gibson/CPEC,(AFB1_A460E),6,(_A_)_(AFB1_A460E2)_(c-),7,A1,B3,B4,A4
4,4,pmas00005,SLIC/Gibson/CPEC,(AFB1_S434E),8,(glutamic_acid)_(AFB1_S434E2)_(-c-),9,A1,B5,B6,A4
5,5,pmas00006,SLIC/Gibson/CPEC,(TIR1_E12K),10,(A)_(TIR1_E12K2)_(c),11,C1,C2,C3,A4
6,6,pmas00007,SLIC/Gibson/CPEC,(TIR1_M473A),12,(C_)_(TIR1_M473A2)_(C1),13,C1,C4,C5,A4
7,7,pmas00008,SLIC/Gibson/CPEC,(TIR1_D170A),14,(c__)_(TIR1_D170A2)_(c---),15,C1,C6,D1,A4
8,8,pmas00009,SLIC/Gibson/CPEC,(AFB1_noflagtag),16,(AFB1_noflagtag2)_(c--),17,C1,D2,D3,A4


In [41]:
#dilvol = ? #this is the volume you bring you stock primer up to in order to make the working dilution

dilvol = (stkprm*stkvol)/dilprm #this is the calculation for volume you bring you stock primer up to based on C1V1=C2V2
H2OAdd = dilvol-stkvol #this is the amount of H2O that you add to your stock primer. since all stock primers are 100uM
                       #concentrations when you start, there is no need to vectorize the H2OAdd variable because it is the
                       #same for each primer.
print (H2OAdd)

39.0


In [42]:
import numpy as np
dilv = np.zeros(len(stktemp)) #initiate a the list of volumes that each stk template in the stktemp vector need to be brought up to. A for loop
          #will be used to solve for each of these.

for i in range(len(stktemp)):
    dilv[i] = (stktemp[i]*stkv)/diltemp #this solves for the volume each template in stktemp needs to be brought up to
print('bring each template up to',dilv, '\n')

H2OAddtemp = np.zeros(len(stktemp))
for i in range(len(stktemp)):
    H2OAddtemp[i] = dilv[i] - stkv
print('to do this add', H2OAddtemp, 'water to each template','\n')


bring each template up to [12.56 19.12 24.08 15.84] 

to do this add [11.56 18.12 23.08 14.84] water to each template 



In [43]:
#PCR Reaction

Amt2addfor1prm = (primerconcentration*pcrvol)/dilprm #this is the amount of primer that you need to add to get the right primer concentration
Amt2addforallprm = Amt2addfor1prm * Numprimers #this is the total amount of primer you will be adding based on number of primers in your rxn
print('you will add', Amt2addfor1prm, 'uL of each primer and', Amt2addforallprm, 'total')
      
Amt2addtemp = (templatengs*pcrvol)/diltemp #this is the amount of template you will add
print('you will add', Amt2addtemp, 'uL of template')

Q5 = pcrvol/2
pcrtotalvolume = Q5 + Amt2addforallprm + Amt2addtemp
print('you will add', Q5, 'uL of Q5')

wawa = pcrvol - pcrtotalvolume
print('add', wawa, 'uL water')

if Amt2addfor1prm < 1:
    print('Dilute primers more in initial dilution')
if Amt2addforallprm > 5:
    print('make primer concentration higher in initial dilution')

you will add 1.0 uL of each primer and 4.0 total
you will add 1.0 uL of template
you will add 12.5 uL of Q5
add 7.5 uL water


In [12]:
def run(protocol: protocol_api.ProtocolContext):
    
    #labware:
    tiprack1 = protocol.load_labware('opentrons_96_tiprack_300ul', '9')
    tiprack2 = protocol.load_labware('opentrons_96_tiprack_20ul','6')
    tuberack1 = protocol.load_labware('opentrons_24_tuberack_generic_2ml_screwcap','1') #holds stock primers and templates
    watertuberack = protocol.load_labware('opentrons_15_tuberack_falcon_15ml_conical','3') #holds molec bio grad H2O
    tuberack2 = protocol.load_labware('opentrons_24_tuberack_nest_1.5ml_snapcap','2') # holds dilute primers and templates
    
    tc_mod = protocol.load_module('Thermocycler Module')
    pcrplate = tc_mod.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
    
    #can put dpnI digestion on either place 6 or 9
    
    
    #pipettes
    right_pipette = protocol.load_instrument('p300_single','right',tipracks=[tiprack1])
    left_pipette = protocol.load_instrument('p20_single','left',tipracks=[tiprack2])
    
    #commands
    
    #add water to template dilution tubes
    left_pipette.pick_up_tip()
    left_pipette.aspirate(29.96, watertuberack['A1'], rate=2.0) #total vol dilute template - vol stock template to add
    left_pipette.dispense(29.96, tuberack2['B1'], rate=2.0)
    #can use the same pipette tip to fill tube B2 since it is only contacting water
    left_pipette.aspirate(9.08, watertuberack['A1'], rate=2.0)
    left_pipette.dispense(9.08, tuberack2['B2'], rate=2.0)
    left_pipette.drop_tip()
    
    #add water to primer dilution tubes
    left_pipette.pick_up_tip()
    left_pipette.aspirate(156, watertuberack['A1'], rate=2.0) #need to put 39uL of water into each dilution tube for primers, so I am aspiration 156 uL and dispensing 39 uL into each tube
    left_pipette.dispense(39, tuberack2['A1'], rate=2.0) #we need to find better way to loop through these commands
    left_pipette.dispense(39, tuberack2['A2'], rate=2.0)
    left_pipette.dispense(39, tuberack2['A3'], rate=2.0)
    left_pipette.dispense(39, tuberack2['A4'], rate=2.0)
    left_pipette.drop_tip()
    
    
    #add stock templates to dilution tubes
    right_pipette.pick_up_tip()
    right_pipette.aspirate(1, tuberack1['D4'], rate=2.0)
    right_pipette.dispense(1, tuberack2['B1'], rate=2.0) #makes a 12.5ng/uL template
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(1, tuberack1['D4'], rate=2.0)
    right_pipette.dispense(1, tuberack2['B2'], rate=2.0) #makes a 12.5ng/uL template
    right_pipette.drop_tip()
    
    #add stock primers to dilution tube - this should be formed into some kind of loop
    right_pipette.pick_up_tip() #add in an iterrows function
    right_pipette.aspirate(1, tuberack1['A1'], rate=2.0) #oligos$well
    right_pipette.dispense(1, tuberack2['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(1, tuberack1['A2'], rate=2.0)
    right_pipette.dispense(1, tuberack2['A2'], rate=2.0)
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(1, tuberack1['A3'], rate=2.0)
    right_pipette.dispense(1, tuberack2['A3'], rate=2.0)
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(1, tuberack1['A4'], rate=2.0)
    right_pipette.dispense(1, tuberack2['A4'], rate=2.0)
    right_pipette.drop_tip()
    
    
    #now mix dilute primers, dilute templates, Q5, and water in pcr tube within thermocycler
    tc_mod.open_lid()
    
    #add water first
    right_pipette.pick_up_tip()
    right_pipette.aspirate(9, watertuberack['A1'], rate=2.0) #need to write a function to add up all volumes that are being added and figure out how much water to add in automated way
    right_pipette.dispense(9, pcrplate['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    #add 2uL of each primer
    right_pipette.pick_up_tip()
    right_pipette.aspirate(2, tuberack2['A1'], rate=2.0)
    right_pipette.dispense(2, pcrplate['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(2, tuberack2['A2'], rate=2.0)
    right_pipette.dispense(2, pcrplate['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(2, tuberack2['A3'], rate=2.0)
    right_pipette.dispense(2, pcrplate['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(2, tuberack2['A4'], rate=2.0)
    right_pipette.dispense(2, pcrplate['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    #add 4uL of each template
    right_pipette.pick_up_tip()
    right_pipette.aspirate(4, tuberack2['B1'], rate=2.0)
    right_pipette.dispense(4, pcrplate['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    right_pipette.pick_up_tip()
    right_pipette.aspirate(4, tuberack2['B2'], rate=2.0)
    right_pipette.dispense(4, pcrplate['A1'], rate=2.0)
    right_pipette.drop_tip()
    
    #Now run thermocycler to amplify DNA    
    tc_mod.close_lid()
    
    tc_mod.set_lid_temperature(105)
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    #put water in for diluted primers (using stkprmh20 variable defined earlier)
    p200.distribute(H2OAdd, bigtuberack.wells('C5'), tuberack2.wells('A1','A2','A3','A4','A5','A6','B1','B2','B3','B4','B5','B6','C1','C2','C3','C4','C5','C6','D1','D2','D3'))  # dilutent
 
    # loop through each tuberack well to dilute stock primers
    for i in range(len(stktemp)):

      # save the source well and destination column to variables
      source = tuberack1.wells(i)
      destination = tuberack2.wells(i)

      # transfer 10uL of source to first well in column (using A variable defined earlier)
      left_pipette.transfer(A, source, destination)

      # dilute the sample down the column (using A variable defined earlier)
      left_pipette.transfer(
        A, column.wells('1', to='6'), column.wells('2', to='4'),
        mix_after=(3, 25))
    
    
    #need to loop through and dilute both of the templates (using variable B defined above)
    p200.distribute(h20totemp, bigtuberack.wells('C5'), tuberack2.wells('D4','D5'))  # dilutent
 
    
    left_pipette.pick_up_tip()
    left_pipette.aspirate(B,tuberack1['D5'])
    left_pipette.dispense(B,tuberack2['D5'])
    left_pipette.drop_tip()
    #########################################################################################
    
    #time to start filling up the pcr tubes
    
    #first add appropriate amount of water
    #put water in for diluted primers (using stkprmh20 variable defined earlier)
    p200.distribute(amth20, bigtuberack.wells('C5'), plate.wells('A1','A2','A3','A4','A5','A6','A7','A8','A9'] # dilutent
 
    
    #move diluted primers into pcr tubes how do i do this with sets of four primers going into each pcr tube
    # loop through each tuberack well to dilute diluted primers
    for i in range(21):

      # save the source well and destination column to variables
      source = tuberack2.wells(i)
      destination = tuberack2.wells(i)

      # transfer 10uL of source to first well in column (using A variable defined earlier)
      left_pipette.transfer(A, source, destination)

      # dilute the sample down the column (using A variable defined earlier)
      left_pipette.transfer(
        A, column.wells('1', to='6'), column.wells('2', to='4'),
        mix_after=(3, 25))
    
      #move diluted templates into pcr tubes - this should probable be done with an if else statement
                                                                 
    
   
    #######################################################################################################
                                                                 
    #set up pcr reaction in the thermocycler module
    
    #open lid
    tc_mod.open_lid()
    
    #close lid
    tc_mod.close_lid()
    
    #set lid temperature
    tc_mod.set_lid_temperature(100)
    
    #set block temp profile
   
    tc_mod.execute_profile(98, hold_time_seconds=30, block_max_volume=50)

    profile = [
    {'temperature': 98, 'hold_time_seconds': 10},
    {'temperature': 65, 'hold_time_seconds': 30},
    {'temperature': 72, }]

    tc_mod.execute_profile(steps=profile, repetitions=34, block_max_volume=50)
    
    tc_mod.execute_profile(72, hold_time_minutes=5, block_max_volume=50)
    
    ################################################################################
    #add DpnI for digestion
    #add DpnI to each PCR product
    
    

    

first we need dilute the stock primers

start off by adding water

In [ ]:
#commands
    
#put water in for diluted primers by using for loop and H2OAdd  
for i in len(oligos['ID Number']):
    
    source = bigtuberack('C5')
    destination = tuberack2.wells(oligos['well'])


left_pipette.pick_up_tip()
        left_pipette.aspirate(H2OAdd,source)
        left_pipette.dispense(H2OAdd,destination)
        left_pipette.drop_tip()

Now add in the stock primer

In [ ]:
for i in len(oligos['ID Number']):
        
        source = tuberack1.wells('well')
        destination = tuberack2.wells('well')
        
        left_pipette.pick_up_tip()
        left_pipette.aspirate(stkvol,source)
        left_pipette.dispense(stkvol,destination)
        left_pipette.drop_tip()
        

Now dilute the template. 

In [ ]:
for i, row in df.iterrows():
    source = tuberack1.wells(df.loc[i,'template_well'])
    destination = tuberack2.wells(df.loc[i,'template_well'])
    
    left_pipette.pick_up_tip()
    left_pipette.aspirate(H2OAddtemp,source)
    left_pipette.dispense(H2OAddtemp,destination)
    left_pipette.drop_tip()

In [ ]:
for i in Rowstemp:
    for j in columnstemp:
        
        source = tuberack1.wells(i+j)
        destination = tuberack2.wells(i+j)
        
        left_pipette.pick_up_tip()
        left_pipette.aspirate(stkv,source)
        left_pipette.dispense(stkv,destination)
        left_pipette.drop_tip()
        

Now we have to set up the PCR reactions with the diluted primers and templates set up in previous steps

In [3]:
from pprint import pprint
import csv

fragid2wells = {}

with open("fragmentid.csv",'r') as infile:
    reader = csv.reader(infile, delimiter =",")
    header = next(reader)
    for row in reader:
        fragid = row[0]
        foligo = row[1]
        roligo = row[2]
        
        if not fragid in fragid2wells: 
            fragid2wells[fragid] = []
            fragid2wells[fragid].append(id2well[foligo])
            fragid2wells[fragid].append(id2well[roligo])

            
pprint(fragid2wells)



{'0': ['A1', 'A2'],
 '1': ['A3', 'A4'],
 '10': ['C1', 'C2'],
 '11': ['C3', 'A4'],
 '12': ['C1', 'C4'],
 '13': ['C5', 'A4'],
 '14': ['C1', 'C6'],
 '15': ['D1', 'A4'],
 '16': ['C1', 'D2'],
 '17': ['D3', 'A4'],
 '2': ['A1', 'A5'],
 '3': ['A6', 'A4'],
 '4': ['A1', 'B1'],
 '5': ['B2', 'A4'],
 '6': ['A1', 'B3'],
 '7': ['B4', 'A4'],
 '8': ['A1', 'B5'],
 '9': ['B6', 'A4']}


In [4]:
Assembly2wells = {}

with open("assemblies.csv",'r') as infile:
    reader = csv.reader(infile, delimiter =",")
    header = next(reader)
    for row in reader:
        assembly = row[0]
        fragid1 = row[1]
        fragid2 = row[2]
        
        if not assembly in Assembly2wells:
            Assembly2wells[assembly] = []
            Assembly2wells[assembly].append(fragid2wells[fragid1]+fragid2wells[fragid2])
            
pprint(Assembly2wells)

{'0': [['A1', 'A2', 'A3', 'A4']],
 '1': [['A1', 'A5', 'A6', 'A4']],
 '2': [['A1', 'B1', 'B2', 'A4']],
 '3': [['A1', 'B3', 'B4', 'A4']],
 '4': [['A1', 'B5', 'B6', 'A4']],
 '5': [['C1', 'C2', 'C3', 'A4']],
 '6': [['C1', 'C4', 'C5', 'A4']],
 '7': [['C1', 'C6', 'D1', 'A4']],
 '8': [['C1', 'D2', 'D3', 'A4']]}


In [5]:
print(Assembly2wells['0'][0])

['A1', 'A2', 'A3', 'A4']


In [ ]:
assemblyrxn2pcrtube = {}
assemblyrxn2pcrtube['0'] = 'A1'
assemblyrxn2pcrtube['1'] = 'A2'
assemblyrxn2pcrtube['2'] = 'A3'
assemblyrxn2pcrtube['3'] = 'A4'
assemblyrxn2pcrtube['4'] = 'A5'
assemblyrxn2pcrtube['5'] = 'A6'
assemblyrxn2pcrtube['6'] = 'A7'
assemblyrxn2pcrtube['7'] = 'A8'
assemblyrxn2pcrtube['8'] = 'A9'
assemblyrxn2pcrtube['9'] = 'A10'

Now I will use these dictionaries to set up the pcr reactions with the appropriate primers, templates, q5, and template.

In [ ]:
assemblyrxns = [0,1,2,3,4,5,6,7,8]
assemblytubes = ['A1','A2','A3','A4','A5','A6','A7','A8','A9']

#put water in each one
for i in assemblytubes:
    source = bigtuberack.wells('C5')
    destination = plate.wells(i)
    
    left_pipette.pick_up_tip()
    left_pipette.aspirate(H2OAdd,source)
    left_pipette.dispense(H2OAdd,destination)
    left_pipette.drop_tip()

#put template in each one


#put primer in each one
for i in assemblyrxns:
    
    listofprimerlocations = Assembly2wells[str(i)][0]
    
    for j in len(Numprimers):
        
        source = tuberack2.wells(listofprimerlocations[j])
        destination = plate.wells(assemblyrxn2pcrtube[str(i)])
        
        left_pipette.pick_up_tip()
        left_pipette.aspirate(Amt2addfor1prm,source)
        left_pipette.aspirate(Amt2addfor1prm,destination)
        left_pipette.drop_tip()

#put Q5 in each one
for i in assemblytubes:
    
    source = ?????????.wells('??')
    destination = plate.wells(i)
    